## Cardiac MRF Workshop 2022-B

This notebook covers what is being done in `set_up_mrf_sim` in `cMRF_Workshop2022_helper`. Here we will load the segmentation and corresponding XML descriptor, look at the MRF signal curves needed for the simulation, load and modify an acquisition template for the data acquisition, define the SNR and coilmaps and run the simulation. 

For this notebook we need:

- an MR rawdata file in ISMRMRD format describing the acquisition process (acquisition template)
- an anatomy segmentation in RAI format (right-anterior-inferior)
- an XML descriptor assigning tissue parameters to the labels.
- contrast information describing the signal over time

In [ ]:
# Import required packages
import shutil, os
from pathlib import Path 

import matplotlib.pyplot as plt

import subprocess

import nibabel as nib
import numpy as np

import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS
import sirf.Gadgetron as pMR
from cil.utilities.jupyter import islicer, link_islicer

import auxiliary_functions as aux
import cMRF_Workshop2022_helper as helper

In [ ]:
# Set main paths
root_path = Path('/home/sirfuser/devel/Data/SA/')
fpath_base = root_path / "Base"

# These folders will be filled with the formatted input 
# for the simulation and will then also contain the output of the simulation
fpath_in = root_path / "Input"
fpath_in.mkdir(exist_ok=True, parents=True)
fpath_out = root_path / "Output"
fpath_out.mkdir(exist_ok=True, parents=True)

In [ ]:
# Tissue parameters (T1, T2, Rho) : xml
fname_xml = fpath_base / 'XCat/XCAT_TissueParameters_XML.xml'
# 3D tissue segmentation : nifti
fpath_segmentation_nii = fpath_base / 'XCat/label_volume_sa.nii'
# Respiratory motion fields : nifti
fpath_resp_mvf = fpath_base / 'XCat/mvf_resp/'
# Cardiac motion fields : nifti
fpath_card_mvf = fpath_base / 'XCat/mvf_card/'

# MR raw k-space data file as acquisition template : ismrmrd
fname_acquisition_template = fpath_base / 'templates/acquisition_template.h5'

# MRF parameters used for EPG simulation
fname_epg_par = fpath_base / 'Fingerprinting/XCAT_tissue_parameter_list.npz'
# MRF signals describing signal behaviour of tissue types in segmentation simulated with EPG
fname_epg_sig = fpath_base / 'Fingerprinting/XCAT_tissue_parameter_fingerprints.npy'
# MRF dictionary for matching and parameter estimation
fname_dict = fpath_base / 'Fingerprinting/dict_70_1500.npz'

# Prefix for ground truth T1, T2 and rho maps
prefix_ground_truth = fpath_out / 'simulation_gt'

## A Clean up folders

When we set up the simulation, we need to save several files temporarily. To make sure we don't run into any problems from previous simulations, we go through all the folders and clean them. When you run the following cell you will be asked if you want to delete any old content. Just say 'y' to agree.

In [ ]:
helper.clear_folders([fpath_in, fpath_out])

## B Prepare segmentation and motion fields

Here we load the segmentation and cardiac/respiratory motion fields and ensure their geometric information is correct. We also select a smaller subset of slices to speed up the simulation. Everything is saved as standard nifti files in `fpath_in`. For more information have a look at the notebook **cMRF_Workshop2022-B**.

In [ ]:
fname_segmentation, fpath_card_mvf_out, fpath_resp_mvf_out = helper.prep_sim(fpath_segmentation_nii, fpath_resp_mvf, fpath_card_mvf, 
                                                                                   fpath_in, num_slices=10, slice_start=35)

## C Segmentation and XML descriptor 

The segmentation is only a 3D volume with a unique index for each tissue type. In order to be able to assign MR parameters (e.g. T1 or T2 times) to it we use an XML descriptor. Here is an example of the XML descriptor. 

It contains
- the label | which voxels it describes in the segmentation
- the name | keeps track of what we actually mean by it 
- MR parameters | T1, T2, proton density and chemical shift
- (PET parameters | only relevant for PET simulations)

<img src="./simulation_xml_example.png" alt="drawing" width=600 />



<h4> Features and Caveats: </h4>

- the XML descriptor **needs to come in the form as displayed**.
- arbitarily many sections of TissueParameter objects can be added, it does not matter if there are parameters described that do not appear in your segmentation.
- **all labels appearing in your segmentations need to appear**, otherwise an error will occur informing you of your mistake. The above example would require a couple of more entries.
- the PET parameters are not taken into consideration for MR simulations, but they still need to be present, fill them with zeros.


## D MRF signal simulation

The MRF signal simulation is not part of the simulation framework but signal curves (i.e. signal behaviour during data acquisition) can be provided by the user. This ensures greater flexibility and new signal models can be easily integrated into the simulation framework. 

Here we load the MRF signals pre-calculated with an EPG simulation and an index list, which maps the signal curves to the different tissue types present in the segmentation. 

In [ ]:
# Re-assign non-unique tissue combinations
epg_input = np.load(fname_epg_par)
inverse_idx = epg_input["unique_idx_inverse"]

# MRF signal curves calculated with EPG
epg_output = np.load(fname_epg_sig)
magnetisation = epg_output[:, inverse_idx]

# Visualise the signal curves for each tissue type
plt.figure()
plt.plot(np.real(epg_output))
plt.xlabel("readout")
plt.ylabel("signal (a.u.)");

## E Acquisition template

In order to tell the simulation how the data was acquired we need a template, i.e. a MR raw data file of the acquisition. Here we use a 2D Golden-radial acquisition. We will load is an `AcquisitionData` object.

In [ ]:
# Load raw data as AcquisitionData object
acquisition_template = pMR.AcquisitionData(str(fname_acquisition_template))

Let's find out a few things about our acquisition template

In [ ]:
# How many readouts?
print(acquisition_template.number_of_readouts())

In [ ]:
# What is the size of k-space
print(acquisition_template.dimensions())

In [ ]:
# What other functionality does the `AcquisitionData` object have?
help(acquisition_template)

In [ ]:
# Here we use only 100 radial lines to speed up the simulation
num_acquisitions = 100
subset_idx = np.arange(num_acquisitions)
acquisition_template = acquisition_template.get_subset(subset_idx)

# We also calculate the trajectory which we will need for the simulation
acquisition_template = pMR.set_goldenangle2D_trajectory(acquisition_template)

## F Create simulation object

Now we can start to put everything together. In order to create a `MRDynamicSimulation` we need

- Segmentation
- XML descriptor

Then we will 

- add acquisition template
- set orientation and position of the slice we want to acquire
- add MRF signal information
- set the SNR
- define the coilmaps


### Create `MRDynamicSimulation`

In [ ]:
# Load the labels into SIRF Nifti Container
segmentation = pReg.NiftiImageData3D(str(fname_segmentation))

# Filename of the XML descriptor
fname_xml = fpath_base / 'XCat/XCAT_TissueParameters_XML.xml'

# Create a simulation object
mrsim = pDS.MRDynamicSimulation(segmentation, str(fname_xml))

### Add the acquisition template

In [ ]:
mrsim.set_template_data(acquisition_template)

### Set orientation and position of the slice

What we want to simulate is of course defined by the segmentation we provided. Nevertheless, we can still define a ROI for our simulated data acquisition within this segmentation. For this we can define an affine transformation (`AffineTransformation`) between segmentation and acquisition. Here we will simply move the central slice a bit

In [ ]:
# Now we set up an affine transformation to move the acquired slice around
offset_x_mm = 0
offset_y_mm = 0
offset_z_mm = -9
rotation_angles_deg = [0,0,0]
translation = np.array([offset_x_mm, offset_y_mm, offset_z_mm])
euler_angles_deg = np.array(rotation_angles_deg)

offset_trafo = pReg.AffineTransformation(translation, euler_angles_deg)
mrsim.set_offset_trafo(offset_trafo)

To verify what we have been doing so far, we will save the current ground truth T1, T2 and M0 maps and visualise them. 

In [ ]:
# We need to set the folder where we want to save the simulated T1, T2 and M0 maps
prefix_ground_truth = fpath_out / "simulation_geometry_acquisition_offset_parametermap"

# Save the ground truth maps
filenames_parametermaps = mrsim.save_parametermap_ground_truth(str(prefix_ground_truth))

# Load and display them
m0_t1_t2_map_gt = helper.load_gt_maps(filenames_parametermaps)
helper.vis_m0_t1_t2_mrf_maps([m0_t1_t2_map_gt,], method_titles=['Ground truth',])

### MRF signal

In [ ]:
# Say which labels are in the dynamic
signal_labels = np.arange(magnetisation.shape[1])
magnetisation = np.transpose(magnetisation[:num_acquisitions,:])

# We need to create an external conrast dynamic
mrf_signal = pDS.ExternalMRSignal(signal_labels, magnetisation)
mrf_dynamic = pDS.ExternalMRContrastDynamic()
mrf_dynamic.add_external_signal(mrf_signal)
mrsim.add_external_contrast_dynamic(mrf_dynamic)

### SNR

Complex Gaussian noise is added to the rawdata after the simulation is performed. The width of the noise distribution is computed based on an SNR parameter and a label of the segmentation for which the SNR is then achieved in image space.

In [ ]:
SNR = 10
SNR_label = 13

mrsim.set_snr(SNR)
mrsim.set_snr_label(SNR_label)

### Coilmaps
The coilmaps used in the simulation can be added flexibly.
The best approach is to first compute them from the rawdata using SIRF and subsequently
- replaced by a simulation of coilmaps that match the data size in the `CoilSensitivityData` object. 
- replaced by a dedicated coilmap which is measured. This will arguably give the best results as they are the most realistic.

It is discouraged to use the computed coil profiles without replacing the data in the `CoilSensitivityData` as anatomy and air which are present in the template acquisitions will be transported into the simulation.

In the auxiliary functions there is an example of how to simulate 2D coilmaps based on using the principal components of Gaussian sensitivity profiles.


In [ ]:
# Get gaussian coilmaps
csm = aux.gaussian_2D_coilmaps(acquisition_template)
csm_arr = csm.as_array()

f, ax = plt.subplots(1,4)
for ic in range(4):
    ax[ic].imshow(np.abs(np.squeeze((csm_arr[ic,...]))), vmin=0.0, vmax=0.35)
    ax[ic].axis('off')
    ax[ic].set_title('coil {}'.format(ic+1))

mrsim.set_csm(csm)

## G Run the simulation

Now we have got everything and we can run the simulation

In [ ]:
mrsim.simulate_data()

### Recap
In this notebook we:

- learned about the XML file describing the tissue parameters,
- added a transformation that defines the 2D slice within the 3D segmentation
- added a raw data file as a template for the data acquisition
- used pre-calculated MRF signals to define the MR signal behaviour during the simulated data acquisition
- defined the SNR and coilmaps for the simulation